Organization:
- Work
  - 1 test: defining functions for part 1, testing on test input
  - 1 run: getting answer for part 1
  - 2 test: ...
  - 2 run: ...
- Utilities: functions I think might help parse general inputs
- Inputs: where I define the test (_t_) and problem (_s_) inputs

# Work

## 1 test

In [134]:
class Monkey():
    def __init__(self, instruction_block):
        self.num_items = 0
        self.ID = int(instruction_block[0].split(':')[0][7:])
        self.start = [int(x) for x in instruction_block[1][18:].split(',')]
        self.operation = instruction_block[2][19:]
        self.test_num = int(instruction_block[3][21:])
        self.test_true = int(instruction_block[4][29:])
        self.test_false = int(instruction_block[5][30:])
        self.items = self.start.copy()
    
    def process_items(self, monkeys):
        for item in self.items:
            old = item
            # Set "new"
            new = eval(self.operation)
            # And now when it gets bored
            new = new // 3
            
            # True
            if new % self.test_num == 0:
                self.give_to_monkey(new, monkeys, self.test_true)
            # False
            else:
                self.give_to_monkey(new, monkeys, self.test_false)
            
            self.num_items += 1
        self.items = []
        
    def give_to_monkey(self, item, monkeys, ID):
        for monkey in monkeys:
            if monkey.ID == ID:
                monkey.add_item(item)
    
    def add_item(self, item):
        self.items += [item]

In [135]:
monkeys = [Monkey(instruction_block) for instruction_block in split(t)]

In [136]:
monkeys[0].__dict__

{'num_items': 0,
 'ID': 0,
 'start': [79, 98],
 'operation': 'old * 19',
 'test_num': 23,
 'test_true': 2,
 'test_false': 3,
 'items': [79, 98]}

In [138]:
for monkey in monkeys:
    monkey.process_items(monkeys)

In [139]:
[monkey.__dict__['items'] for monkey in monkeys]

[[20, 23, 27, 26], [2080, 25, 167, 207, 401, 1046], [], []]

In [140]:
# 20 rounds
monkeys = [Monkey(instruction_block) for instruction_block in split(t)]
for i in range(20):
    for monkey in monkeys:
        monkey.process_items(monkeys)

In [141]:
[monkey.num_items for monkey in monkeys]

[101, 95, 7, 105]

In [143]:
101 * 105

10605

## 1 run

In [144]:
monkeys = [Monkey(instruction_block) for instruction_block in split(s)]
for i in range(20):
    for monkey in monkeys:
        monkey.process_items(monkeys)

In [145]:
[monkey.num_items for monkey in monkeys]

[290, 59, 222, 342, 78, 302, 347, 318]

In [146]:
sorted([monkey.num_items for monkey in monkeys])

[59, 78, 222, 290, 302, 318, 342, 347]

In [147]:
342 * 347

118674

## 2 test

Taking item worries modulo the LCM of the checking moduli of the monkeys suffices for the worry computations

In [186]:
# Just redefine to not divide worry by 3
# But this makes the worry levels blow up! All that matters though is the modulus, so when giving an item we'll reduce by that

class Monkey():
    def __init__(self, instruction_block):
        self.num_items = 0
        self.ID = int(instruction_block[0].split(':')[0][7:])
        self.start = [int(x) for x in instruction_block[1][18:].split(',')]
        self.operation = instruction_block[2][19:]
        self.test_num = int(instruction_block[3][21:])
        self.test_true = int(instruction_block[4][29:])
        self.test_false = int(instruction_block[5][30:])
        self.items = self.start.copy()
    
    def process_items(self, monkeys, lcm):
        for item in self.items:
            old = item
            # Set "new"
            new = eval(self.operation)
            
            # True
            if new % self.test_num == 0:
                self.give_to_monkey(new, monkeys, self.test_true, lcm)
            # False
            else:
                self.give_to_monkey(new, monkeys, self.test_false, lcm)
            
            self.num_items += 1
        self.items = []
        
    def give_to_monkey(self, item, monkeys, ID, lcm):
        for monkey in monkeys:
            if monkey.ID == ID:
                monkey.add_item(item, lcm)
    
    def add_item(self, item, lcm):
        self.items += [item % lcm]

In [198]:
monkeys = [Monkey(instruction_block) for instruction_block in split(t)]

In [199]:
import numpy as np

In [200]:
[monkey.test_num for monkey in monkeys]

[23, 19, 13, 17]

In [201]:
lcm = 1
for monkey in monkeys:
    lcm = np.lcm(lcm, monkey.test_num)
lcm = int(lcm)
lcm

96577

In [202]:
for i in range(10000):
    for monkey in monkeys:
        monkey.process_items(monkeys, lcm)

In [203]:
[monkey.num_items for monkey in monkeys]

[52166, 47830, 1938, 52013]

In [204]:
sorted([monkey.num_items for monkey in monkeys])

[1938, 47830, 52013, 52166]

In [205]:
52013 * 52166

2713310158

## 2 run

In [206]:
monkeys = [Monkey(instruction_block) for instruction_block in split(s)]

lcm = 1
for monkey in monkeys:
    lcm = np.lcm(lcm, monkey.test_num)
lcm = int(lcm)
lcm

for i in range(10000):
    for monkey in monkeys:
        monkey.process_items(monkeys, lcm)

In [207]:
sorted([monkey.num_items for monkey in monkeys])

[27035, 71968, 89792, 153007, 161890, 162012, 179690, 179940]

In [208]:
179690 * 179940

32333418600

# Utilities

In [4]:
# Remove initial/final \n characters
def clean(s):
    return s[1:-1]

# Split at \n characters
# If there are \n\n characters, split into blocks too
def split(s):
    out = [block.split('\n') for block in clean(s).split('\n\n')]
    if len(out) == 1:
        return out[0]
    else:
        return out

# Apply a function(s) to a list or "block" data (2-level list)
def apply_func(data, func, nested=False):
    if not isinstance(func, list):
        func = [func]
        
    def _func(x):
        for f in func:
            x = f(x)
        return x
        
    if nested:
        return [[_func(x) for x in block] for block in data]
    else:
        return [_func(x) for x in data]

# Split, parsing everything as ints
def split_int(s):
    return apply_func(split(s), int)

# Split, parsing everything as float
def split_float(s):
    return apply_func(split(s), float)

# Inputs

In [1]:
t = """
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1
"""

In [2]:
s = """
Monkey 0:
  Starting items: 85, 79, 63, 72
  Operation: new = old * 17
  Test: divisible by 2
    If true: throw to monkey 2
    If false: throw to monkey 6

Monkey 1:
  Starting items: 53, 94, 65, 81, 93, 73, 57, 92
  Operation: new = old * old
  Test: divisible by 7
    If true: throw to monkey 0
    If false: throw to monkey 2

Monkey 2:
  Starting items: 62, 63
  Operation: new = old + 7
  Test: divisible by 13
    If true: throw to monkey 7
    If false: throw to monkey 6

Monkey 3:
  Starting items: 57, 92, 56
  Operation: new = old + 4
  Test: divisible by 5
    If true: throw to monkey 4
    If false: throw to monkey 5

Monkey 4:
  Starting items: 67
  Operation: new = old + 5
  Test: divisible by 3
    If true: throw to monkey 1
    If false: throw to monkey 5

Monkey 5:
  Starting items: 85, 56, 66, 72, 57, 99
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 1
    If false: throw to monkey 0

Monkey 6:
  Starting items: 86, 65, 98, 97, 69
  Operation: new = old * 13
  Test: divisible by 11
    If true: throw to monkey 3
    If false: throw to monkey 7

Monkey 7:
  Starting items: 87, 68, 92, 66, 91, 50, 68
  Operation: new = old + 2
  Test: divisible by 17
    If true: throw to monkey 4
    If false: throw to monkey 3
"""